<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/Spark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pyspark and customize Colab configuration


The pyspark installation will persist until the runtime is recycled.

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install ucimlrepo

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,654 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,450 kB]


'/usr/local/lib/python3.10/dist-packages/pyspark'

# Initialize Spark

In [2]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

sc = spark.sparkContext
spark

# Load Bank Dataset

In [7]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

# fetch dataset
bank_marketing = fetch_ucirepo(id=222)

# data (as pandas dataframes)
dataset_X = bank_marketing.data.features
dataset_y = bank_marketing.data.targets

dataset = pd.concat([dataset_X, dataset_y], axis=1)
dataset.rename(columns={"y": "deposit"}, inplace=True)

sdf = spark.createDataFrame(dataset)
sdf.createOrReplaceTempView("bank_data")

In [8]:
sdf.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
|age|         job| marital|education|default|balance|housing|loan|contact|day_of_week|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------------+--------+---------+-------+-------+-------+----+-------+-----------+-----+--------+--------+-----+--------+--------+-------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|    NaN|          5|  may|     261|       1|   -1|       0|     NaN|     no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|    NaN|          5|  may|     151|       1|   -1|       0|     NaN|     no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|    NaN|          5|  may|      76|       1|   -1|       0|     NaN|     no|
| 47| blue-collar| married|      NaN|     no|   1506|    yes|  no|    NaN|          5|  may|      92|       1|   -1|       0|     NaN|     no|

In [5]:
# Define python function to calculate power to weight ratio
def get_power_weight_ratio(hp, weight):
  return float(hp) / float(weight)

# Register UDF to be used with SQL
spark.udf.register("get_power_weight_ratio", get_power_weight_ratio, T.DoubleType())

# Using UDF to create a new column and sorting in descending orders
spark.sql("SELECT *, ROUND(get_power_weight_ratio(Horsepower, Weight), 4) AS power_weight_ratio FROM cars ORDER BY power_weight_ratio DESC").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Horsepower` cannot be resolved. Did you mean one of the following? [`deposit`, `poutcome`, `previous`, `age`, `balance`].; line 1 pos 39;
'Sort ['power_weight_ratio DESC NULLS LAST], true
+- 'Project [age#0L, job#1, marital#2, education#3, default#4, balance#5L, housing#6, loan#7, contact#8, day_of_week#9L, month#10, duration#11L, campaign#12L, pdays#13L, previous#14L, poutcome#15, deposit#16, 'ROUND('get_power_weight_ratio('Horsepower, 'Weight), 4) AS power_weight_ratio#103]
   +- SubqueryAlias cars
      +- View (`cars`, [age#0L,job#1,marital#2,education#3,default#4,balance#5L,housing#6,loan#7,contact#8,day_of_week#9L,month#10,duration#11L,campaign#12L,pdays#13L,previous#14L,poutcome#15,deposit#16])
         +- LogicalRDD [age#0L, job#1, marital#2, education#3, default#4, balance#5L, housing#6, loan#7, contact#8, day_of_week#9L, month#10, duration#11L, campaign#12L, pdays#13L, previous#14

# Supervised Machine Learning

In [9]:
# Checking the data imbalance
sdf.groupBy('deposit').agg(F.count('*').alias("count")).show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no|39922|
|    yes| 5289|
+-------+-----+



In [10]:
# Stratified sampling to balance the dataset
fractions = {
    "no": 0.15,
    "yes": 1.0
}

sdf = sdf.sampleBy("deposit", fractions, seed=42)

In [11]:
# Checking the data imbalance
sdf.groupBy('deposit').agg(F.count('*').alias("count")).show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no| 5973|
|    yes| 5289|
+-------+-----+



### Summary statistics for numeric variables

In [12]:
# statistics for all columns with numerical variables

numeric_features = [t[0] for t in sdf.dtypes if t[1] in ['int', 'bigint', 'float', 'double']]
sdf.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,11262,41.34833954892559,11.886038147594325,18,95
balance,11262,1544.8779968034096,3231.137341840109,-8019,81204
day_of_week,11262,15.488012786361216,8.377376396186428,1,31
duration,11262,369.2394778902504,345.2136721708763,0,3881
campaign,11262,2.513141537915113,2.664956670363362,1,43
pdays,11262,50.23654768247203,107.10815307320459,-1,854
previous,11262,0.8019889895222874,2.209449144538357,0,58


 - day and month columns are not really useful, we will remove these two columns.

In [13]:
sdf = sdf.select(
    'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
    'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit'
)
cols = sdf.columns
sdf.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



### Preparing Dataset for Machine Learning

The process includes
* Category Indexing,
* One-Hot Encoding and
* VectorAssembler (a feature transformer that merges multiple columns into a vector column)

This code indexes each categorical column using the StringIndexer, then converts the indexed categories into one-hot encoded variables.

The resulting output has the binary vectors appended to the end of each row. We use the StringIndexer again to encode our labels to label indices.

Next, we use the VectorAssembler to combine all the feature columns into a single vector column.

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stages = []
categoricalColumns = [
    'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome'
]

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(
        inputCols=[stringIndexer.getOutputCol()],
        outputCols=[categoricalCol + "classVec"]
    )
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [21]:
stages

[StringIndexer_5ce47008fd69,
 OneHotEncoder_54ef661dd18f,
 StringIndexer_77396e2aee83,
 OneHotEncoder_0c5bf2b8981f,
 StringIndexer_e64e39f9412f,
 OneHotEncoder_e30909ce90a6,
 StringIndexer_b8eddfd0e0ee,
 OneHotEncoder_005aa2c9d103,
 StringIndexer_2937926a58ce,
 OneHotEncoder_351e22a5cfeb,
 StringIndexer_083044ec3d3c,
 OneHotEncoder_f9698e7e91c6,
 StringIndexer_4ae7598e4799,
 OneHotEncoder_38d0b1eafa82,
 StringIndexer_14acbfd30d4a,
 OneHotEncoder_82044b8dd6d9,
 StringIndexer_aa0630451cfb,
 VectorAssembler_5fde1c5df23e]

### Pipeline

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(sdf)
sdf = pipelineModel.transform(sdf)
selectedCols = ['label', 'features'] + cols
sdf = sdf.select(selectedCols)
sdf.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



### Split dataset into train and test set

In [16]:
train, test = sdf.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7851
Test Dataset Count: 3411


### Random Forest Classifier

In [17]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
rfPreds = rfModel.transform(test)
rfPreds.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
|age|       job|label|       rawPrediction|prediction|         probability|
+---+----------+-----+--------------------+----------+--------------------+
| 43|management|  0.0|[16.9351726513819...|       0.0|[0.84675863256909...|
| 34|management|  0.0|[16.4043645336822...|       0.0|[0.82021822668411...|
| 36|management|  0.0|[15.1300019038857...|       0.0|[0.75650009519428...|
| 41|management|  0.0|[16.4766031308205...|       0.0|[0.82383015654102...|
| 57|management|  0.0|[16.2459223107413...|       0.0|[0.81229611553706...|
| 31|management|  0.0|[16.0345449835063...|       0.0|[0.80172724917531...|
| 37|management|  0.0|[17.3418536903619...|       0.0|[0.86709268451809...|
| 44|management|  0.0|[16.8915026385632...|       0.0|[0.84457513192816...|
| 45|management|  0.0|[8.63884214315815...|       1.0|[0.43194210715790...|
| 47|management|  0.0|[16.7745064305459...|       0.0|[0.83872532152729...|
+---+-------

#### Evaluate the Random Forest Classifier

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

rfEval = BinaryClassificationEvaluator()
rfROC = rfEval.evaluate(rfPreds, {rfEval.metricName: "areaUnderROC"})
print("Test Area Under ROC: " + str(rfROC))

Test Area Under ROC: 0.8905965426896236


### Gradient-Boosted Tree Classifier

In [19]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
gbtPreds = gbtModel.transform(test)
gbtPreds.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+----------+-----+--------------------+----------+--------------------+
|age|       job|label|       rawPrediction|prediction|         probability|
+---+----------+-----+--------------------+----------+--------------------+
| 43|management|  0.0|[1.29163386920042...|       0.0|[0.92977692547934...|
| 34|management|  0.0|[1.03096110830084...|       0.0|[0.88714676049183...|
| 36|management|  0.0|[0.76660223502044...|       0.0|[0.82247468273028...|
| 41|management|  0.0|[0.57671821499725...|       0.0|[0.76013803504645...|
| 57|management|  0.0|[1.23475128779114...|       0.0|[0.92197598952629...|
| 31|management|  0.0|[1.12123782481789...|       0.0|[0.90399952101299...|
| 37|management|  0.0|[1.32064356049446...|       0.0|[0.93347194192069...|
| 44|management|  0.0|[1.13329281791413...|       0.0|[0.90607160800932...|
| 45|management|  0.0|[-0.5381452071238...|       1.0|[0.25420866301427...|
| 47|management|  0.0|[1.07923290374290...|       0.0|[0.89645722882976...|
+---+-------

#### Evaluate the Gradient-Boosted Tree Classifier

In [20]:
gbtEval = BinaryClassificationEvaluator()
gbtROC = gbtEval.evaluate(gbtPreds, {gbtEval.metricName: "areaUnderROC"})
print("Test Area Under ROC: " + str(gbtROC))

Test Area Under ROC: 0.8992009144210671




---

